Import needed libraries

In [1]:
# COMMON
from bip_utils import Bip39MnemonicGenerator, Bip39SeedGenerator

# ETH
from bip_utils import Bip32Slip10Secp256k1
from Crypto.Hash import keccak

# Needs module `pip install bip-utils`

Step 1: Generate a Mnemonic
A mnemonic phrase is a sequence of 12, 15, 18, 21, or 24 words. We'll use the Bip39MnemonicGenerator class to create one.

We prompt user to input mnemonic or generate a new one. Enter 'new' without quotes to generate a new mnemonic or enter the 12 words to use them as mnemonic instead of generating.

In [2]:
# Step 1: Generate a 12-word mnemonic
generate_new = input("Enter mnemonic you want to use or enter 'new' without quotes or leave empty to generate a random one.")
if generate_new == 'new' or generate_new == '':
  mnemonic = Bip39MnemonicGenerator().FromWordsNumber(12)
else:
  mnemonic = generate_new

print(f"Mnemonic: {mnemonic}")

Mnemonic: razor angle rib avoid alarm they picture define ship wink describe hire


Step 2: Convert Mnemonic to Seed.
The mnemonic needs to be converted into a seed, which is a 512-bit number.

In [3]:
# Step 2: Generate the seed from the mnemonic
seed = Bip39SeedGenerator(mnemonic).Generate()
print(f"Mnemonic: {mnemonic}")
print(f"Seed: {seed.hex()}")

Mnemonic: razor angle rib avoid alarm they picture define ship wink describe hire
Seed: 0e5d8859ef71ccff75b9ada5d9c22b2df947b5ba9aca6fb0ca7bb7f25c78ae49f04eb80dafa8ed9a11e8f4f0cb27c694031f610d0908d4fdd5ba4054e56464aa


Step 3: To get n-th key for the ethereum key, one mnemonic can generate an infinite number of key pairs.

This is achieved through the use of hierarchical deterministic (HD) wallets, which follow a tree structure. Each node in the tree can generate child nodes (key pairs) based on a specific path.

To derive the nth key pair and its corresponding Ethereum address, we modify the derivation path. Typically, the path follows the format:

`m / purpose' / coin_type' / account' / change / address_index`

For Ethereum, a commonly used path is:
`m / 44' / 60' / 0' / 0 / n`

Here, n is the index of the address you want to derive. Let's update the script to allow for deriving the nth key pair and address:

In [4]:
# Create a BIP-32 master key from the seed
bip32_ctx = Bip32Slip10Secp256k1.FromSeed(seed)

# Get the n-th number we want to generate
index_key = input("Enter the key number you want to generate")
if index_key == '':
  index_key = 0

# Derive the nth account private key (m/44'/60'/0'/0/n)
eth_account_path = f"m/44'/60'/0'/0/{index_key}"
private_key = bip32_ctx.DerivePath(eth_account_path).PrivateKey().Raw().ToHex()
print(f"Private Key: {private_key}")

# Derive the public key from the private key
public_key = bip32_ctx.DerivePath(eth_account_path).PublicKey().RawUncompressed().ToHex()
print(f"Public Key: {public_key}")

# Derive the Ethereum address from the public key
# Remove the '0x04' prefix (which indicates an uncompressed public key)
public_key_bytes = bytes.fromhex(public_key)[1:]

# Compute the Keccak-256 hash of the public key
keccak_hash = keccak.new(digest_bits=256)
keccak_hash.update(public_key_bytes)
public_key_hash = keccak_hash.digest()

# Take the last 20 bytes of the hash as the Ethereum address
eth_address = public_key_hash[-20:].hex()
eth_address = "0x" + eth_address
print(f"Ethereum Address: {eth_address}")

Private Key: 8a710c3e39b329b03393e83d1c58b9c96433c17adfe22b8f019d8bde46ef5f6d
Public Key: 040198a483c7e3863cd0062aae1623dd3cf04024a143dba73017196e5a63de976f872c7bb40782b47ab2aeba651651673aab4cd45dfafd48fdb3cac28350afb67c
Ethereum Address: 0xbe56499edd3994e15e862f87e2898e4898ad3f83


# ----------------------
### Bitcoin
#### Legacy Generate bitcoin address from the same mnemonic

In [7]:
#BTC
from bip_utils import Bip44, Bip44Coins, Bip44Changes

# Get the n-th number we want to generate
btc_index_key = input("Enter the BTC key number you want to generate")
if btc_index_key == '':
  btc_index_key = 0

# Create a BIP-44 object for Bitcoin
bip44_mst_ctx = Bip44.FromSeed(seed, Bip44Coins.BITCOIN)

# Derive the nth account key pair (m/44'/0'/0'/0/n)
bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(btc_index_key)

# Get the private key
private_key = bip44_acc_ctx.PrivateKey().Raw().ToHex()

# Get the public key
public_key = bip44_acc_ctx.PublicKey().RawCompressed().ToHex()

# Get the Bitcoin address
bitcoin_address = bip44_acc_ctx.PublicKey().ToAddress()

print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")
print(f"Legacy Bitcoin Address: {bitcoin_address}")


Private Key: 3cea4719a1f0b99897fafd4602978bec5f773a59e4f2462039b002b731ffc52c
Public Key: 02ef1b97d4440df956855f4991348ef56b75b65f3f8c208ffada79b304b0b77140
Legacy Bitcoin Address: 1GUXGKfSj2yaa4xi5MXAS9UHZsJybVrkUh


Segwit BTC Addresses

In [26]:
# BTC Segwit
from bip_utils import Bip84, Bip84Coins

# Create a BIP-84 object for Bitcoin (native SegWit)
bip84_mst_ctx = Bip84.FromSeed(seed, Bip84Coins.BITCOIN)

# Derive the nth account key pair (m/84'/0'/0'/0/n)
bip84_acc_ctx = bip84_mst_ctx.Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(btc_index_key)

# Get the private key
private_key = bip84_acc_ctx.PrivateKey().Raw().ToHex()

# Get the public key
public_key = bip84_acc_ctx.PublicKey().RawCompressed().ToHex()

# Get the Bitcoin SegWit address
bitcoin_address = bip84_acc_ctx.PublicKey().ToAddress()

print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")
print(f"Bitcoin SegWit Address: {bitcoin_address}")

Private Key: c7465e1415688012fd170b7991614e11ebc63bdcc449b28e115c329b55d6b87e
Public Key: 02710cad6d8dac03473e734245fa27cf9ff228191ffc9e17fe2dd2df9b0c2645a2
Bitcoin SegWit Address: bc1qdta3uvvueskkxwkuw2jql4qyz6nujku59jx0ew


# ------------
#### Dogecoin

In [27]:
# Get the n-th number we want to generate
doge_index_key = input("Enter the DOGE key number you want to generate")
if doge_index_key == '':
  doge_index_key = 0

# Create a BIP-44 object for Dogecoin
bip44_mst_ctx = Bip44.FromSeed(seed, Bip44Coins.DOGECOIN)

# Derive the nth account key pair (m/44'/3'/0'/0/n)
bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(doge_index_key)

# Get the private key
private_key = bip44_acc_ctx.PrivateKey().Raw().ToHex()

# Get the public key
public_key = bip44_acc_ctx.PublicKey().RawCompressed().ToHex()

# Get the Dogecoin address
dogecoin_address = bip44_acc_ctx.PublicKey().ToAddress()

print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")
print(f"Dogecoin Address: {dogecoin_address}")

Private Key: 1284c4d6c5832da5a97fd2e8cbd01f825c89a766c883cf5110efb79149da4af8
Public Key: 032821c7380e0338890a4dfaf5394cad1b5c957a8d1fb4a187e98baca503486e88
Dogecoin Address: DCaDeocPKHnH1yyrYQNKyFHd4YdBz64Yfg


# -----------
#### Litecoin legacy

In [28]:
# Get the n-th number we want to generate
lite_index_key = input("Enter the Litecoin key number you want to generate")
if lite_index_key == '':
  lite_index_key = 0

# Create a BIP-44 object for Litecoin
bip44_mst_ctx = Bip44.FromSeed(seed, Bip44Coins.LITECOIN)

# Derive the nth account key pair (m/44'/2'/0'/0/n)
bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(lite_index_key)

# Get the private key
private_key = bip44_acc_ctx.PrivateKey().Raw().ToHex()

# Get the public key
public_key = bip44_acc_ctx.PublicKey().RawCompressed().ToHex()

# Get the Litecoin address
litecoin_address = bip44_acc_ctx.PublicKey().ToAddress()

print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")
print(f"Litecoin Address: {litecoin_address}")

Private Key: 139561d621707601e4d0b63b47897dcb0bd17abfdd1c561e465bded63f53c4ea
Public Key: 02d1f3bb311adb4f7bce224983ebbb8d45a332718424859f338fb767b1c6ddf29f
Litecoin Address: LKNuBXHhBjwLDqGiCBqPgdSPry13T6nT3c


#### Litecoin segwit

In [29]:
# Litecoin Segwit
from bip_utils import Bip84, Bip84Coins

# Create a BIP-84 object for Litecoin (native SegWit)
bip84_mst_ctx = Bip84.FromSeed(seed, Bip84Coins.LITECOIN)

# Derive the nth account key pair (m/84'/0'/0'/0/n)
bip84_acc_ctx = bip84_mst_ctx.Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(lite_index_key)

# Get the private key
private_key = bip84_acc_ctx.PrivateKey().Raw().ToHex()

# Get the public key
public_key = bip84_acc_ctx.PublicKey().RawCompressed().ToHex()

# Get the Litecoin SegWit address
bitcoin_address = bip84_acc_ctx.PublicKey().ToAddress()

print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")
print(f"Litecoin SegWit Address: {bitcoin_address}")

Private Key: 6f83d3e0ccfe105124904dbf333be574696539bf27d6ee83ac52a0056ad9fa0b
Public Key: 027e636996a7d0fe0d9383491336427aaaffb69185a8f7aa79a29475c16de921a8
Litecoin SegWit Address: ltc1qrc835vwnz0jvr2jupy88uzndf7m30r425n60zy


# ------------
#### Solana --- Failing

In [30]:
# Get the n-th number we want to generate
sol_index_key = input("Enter the Solana key number you want to generate")
if sol_index_key == '':
  sol_index_key = 0

# Create a BIP-44 object for Solana
bip44_mst_ctx = Bip44.FromSeed(seed, Bip44Coins.SOLANA)

# Derive the nth account key pair (m/44'/2'/0'/0/n)
bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(sol_index_key)

# Get the private key
private_key = bip44_acc_ctx.PrivateKey().Raw().ToHex()

# Get the public key
public_key = bip44_acc_ctx.PublicKey().RawCompressed().ToHex()

# Get the Litecoin address
solana_address = bip44_acc_ctx.PublicKey().ToAddress()

print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")
print(f"Solana Address: {solana_address}")

Private Key: 96dbb76c2fef46163236fefc5a6733c679de8a4e8bc37f5108bcb1f108ebe673
Public Key: 00c2c3df4f8dff493f82cf2bc11778f1998ff577b7d1909c1e9dda242c12a2e288
Solana Address: E7HKnMduPbnXxDQupMHQSi4muzYb9zgpGgs6ZPNfnTXD
